In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target']

print(X.shape, y.shape)


(70000, 784) (70000,)


In [ ]:
#70000 images of 784 pixels each = 28x28
#60000 training images
#10000 test images
